# Model Onnx Parser

From this section, let's start looking at [fpgaconvnet-model](https://github.com/AlexMontgomerie/fpgaconvnet-model), which converts the onnx graph to a hardware accelerator configuration, providing estimated metrics with the help of performance and resource modeling.

The entry point of fpgaconvnet-model is the [`Parser`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/parser/Parser.py#L26) that translates onnx to our own Intermediate Representation as Synchronous Data Flow (SDF) graph which is built on the [networkx](https://networkx.org/) package.

Now either use the onnx file generated in the previous tutorial or use the one we provide, run the following code which will return a networkx graph. 

In [1]:
from fpgaconvnet.parser.Parser import Parser

onnx_path = "fp16/vgg16_bn.onnx"

parser = Parser(custom_onnx=True, batch_size=1)
net = parser.onnx_to_fpgaconvnet(onnx_path)

CRITICAL WARNING: node Flatten_31 is skipped in hardware


The generated networkx graph has the hierarchy of:

* [`Network`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/models/network/Network.py), corresponds to the whole onnx graph.
* [`Partition`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/models/partition/Partition.py), a subgraph that eventually will be converted to a bitstream. partitions can be either placed across multi-FPGA devices or scheduled on a single divice through reconfiguration.
* [`Layer`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/models/layers/Layer.py), corresponds to a node in onnx, and the mapping between them can be found [here](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/parser/Parser.py#L150). 
* [`Module`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/models/modules/Module.py), corresponds to the hardware module implemented in [fpgaconvnet-hls](https://github.com/AlexMontgomerie/fpgaconvnet-hls/tree/main/fpgaconvnet/hls/hardware).

For example, to iterative over the generated networkx graph, please run:

In [2]:
for i, partition in enumerate(net.partitions):
    print(i, partition)
    for j, layer_name in enumerate(partition.graph.nodes):
        layer_type = partition.graph.nodes[layer_name]["type"]
        layer = partition.graph.nodes[layer_name]["hw"]
        print(j, layer_name, layer_type)

0 <fpgaconvnet.models.partition.Partition.Partition object at 0x7fccc778b820>
0 Conv_0 LAYER_TYPE.Convolution
1 Relu_1 LAYER_TYPE.ReLU
2 Conv_2 LAYER_TYPE.Convolution
3 Relu_3 LAYER_TYPE.ReLU
4 MaxPool_4 LAYER_TYPE.Pooling
5 Conv_5 LAYER_TYPE.Convolution
6 Relu_6 LAYER_TYPE.ReLU
7 Conv_7 LAYER_TYPE.Convolution
8 Relu_8 LAYER_TYPE.ReLU
9 MaxPool_9 LAYER_TYPE.Pooling
10 Conv_10 LAYER_TYPE.Convolution
11 Relu_11 LAYER_TYPE.ReLU
12 Conv_12 LAYER_TYPE.Convolution
13 Relu_13 LAYER_TYPE.ReLU
14 Conv_14 LAYER_TYPE.Convolution
15 Relu_15 LAYER_TYPE.ReLU
16 MaxPool_16 LAYER_TYPE.Pooling
17 Conv_17 LAYER_TYPE.Convolution
18 Relu_18 LAYER_TYPE.ReLU
19 Conv_19 LAYER_TYPE.Convolution
20 Relu_20 LAYER_TYPE.ReLU
21 Conv_21 LAYER_TYPE.Convolution
22 Relu_22 LAYER_TYPE.ReLU
23 MaxPool_23 LAYER_TYPE.Pooling
24 Conv_24 LAYER_TYPE.Convolution
25 Relu_25 LAYER_TYPE.ReLU
26 Conv_26 LAYER_TYPE.Convolution
27 Relu_27 LAYER_TYPE.ReLU
28 Conv_28 LAYER_TYPE.Convolution
29 Relu_29 LAYER_TYPE.ReLU
30 GlobalMaxPool_

The `net` object has a single partition by default, which can also be accessed by `net.partitions[0]`. Regarding the layers, `partition.graph.nodes[layer_name]` is a dict as defined [`here`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/parser/onnx/parse.py#L67).

In [3]:
net.partitions[0].graph.nodes["Conv_0"].keys()

dict_keys(['type', 'onnx_node', 'onnx_input', 'onnx_output', 'attr', 'hw', 'inputs'])

Its `"type"` is an enumeration object which is defined [`here`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/tools/layer_enum.py#L6).

In [4]:
net.partitions[0].graph.nodes["Conv_0"]["type"]

<LAYER_TYPE.Convolution: 4>

while the actual `Layer` object can be accessed by `"hw"`,

In [5]:
net.partitions[0].graph.nodes["Conv_0"]["hw"]

ConvolutionLayer(_rows=32, _cols=32, _channels=3, _coarse_in=1, _coarse_out=1, input_compression_ratio=[1.0, 1.0, 1.0], output_compression_ratio=[1.0], mem_bw_in=100.0, mem_bw_out=100.0, data_t=FixedPoint(width=16, binary_point=8), buffer_depth=2, modules=OrderedDict([('pad', Pad(rows=32, cols=32, channels=3, data_width=16, rsc_coef={'FF': [], 'LUT': [], 'DSP': [], 'BRAM': []}, pad_top=1, pad_bottom=1, pad_left=1, pad_right=1, backend='chisel', regression_model='linear_regression', streams=1, latency_mode=False, block=False)), ('sliding_window', SlidingWindow(rows=34, cols=34, channels=3, data_width=16, rsc_coef={'Logic_LUT': array([6.46840521, 0.        , 0.        , 3.92659388, 2.45084112]), 'LUT_RAM': array([ 0.07302216,  0.        , 14.23619095]), 'LUT_SR': array([0.]), 'FF': array([ 0.        ,  0.        , 20.77153538,  0.        ,  0.        ,
        0.        ,  5.21236925,  3.44020551,  0.        ,  0.        ]), 'DSP': array([0.]), 'BRAM36': array([0.]), 'BRAM18': array([0.]

and the `Module`s inside the `Layer` can be accessed by

In [6]:
net.partitions[0].graph.nodes["Conv_0"]["hw"].modules

OrderedDict([('pad',
              Pad(rows=32, cols=32, channels=3, data_width=16, rsc_coef={'FF': [], 'LUT': [], 'DSP': [], 'BRAM': []}, pad_top=1, pad_bottom=1, pad_left=1, pad_right=1, backend='chisel', regression_model='linear_regression', streams=1, latency_mode=False, block=False)),
             ('sliding_window',
              SlidingWindow(rows=34, cols=34, channels=3, data_width=16, rsc_coef={'Logic_LUT': array([6.46840521, 0.        , 0.        , 3.92659388, 2.45084112]), 'LUT_RAM': array([ 0.07302216,  0.        , 14.23619095]), 'LUT_SR': array([0.]), 'FF': array([ 0.        ,  0.        , 20.77153538,  0.        ,  0.        ,
                      0.        ,  5.21236925,  3.44020551,  0.        ,  0.        ]), 'DSP': array([0.]), 'BRAM36': array([0.]), 'BRAM18': array([0.])}, kernel_size=[3, 3], stride=[1, 1], pad_top=0, pad_right=0, pad_bottom=0, pad_left=0, backend='chisel', regression_model='linear_regression', streams=1)),
             ('squeeze',
              Sque

which is a dict with module name as the key and module object as the value. To learn more about the [`Network`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/models/network/Network.py), [`Partition`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/models/partition/Partition.py), [`Layer`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/models/layers/Layer.py), and [`Module`](https://github.com/AlexMontgomerie/fpgaconvnet-model/blob/dev-petros/fpgaconvnet/models/modules/Module.py), please continue the rest of the tutorial.